In [45]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("MinTemp").getOrCreate()

In [71]:
# read csv
daily_temp = spark.read.csv(
    "../data/1800.csv",
    inferSchema=True,
)
# we only want the first 4 columns
daily_temp = daily_temp.select(["_c0", "_c1", "_c2", "_c3"])
# rename columns
column_names = ["station_id", "date", "measurement_type", "measurement_value"]
daily_temp = daily_temp.toDF(*column_names)
# Convert 'date' column from string to date
daily_temp = daily_temp.withColumn("date", F.to_date(daily_temp["date"], "yyyyMMdd"))

daily_temp.show(5)

+-----------+----------+----------------+-----------------+
| station_id|      date|measurement_type|measurement_value|
+-----------+----------+----------------+-----------------+
|ITE00100554|1800-01-01|            TMAX|              -75|
|ITE00100554|1800-01-01|            TMIN|             -148|
|GM000010962|1800-01-01|            PRCP|                0|
|EZE00100082|1800-01-01|            TMAX|              -86|
|EZE00100082|1800-01-01|            TMIN|             -135|
+-----------+----------+----------------+-----------------+
only showing top 5 rows



In [72]:
# we only want the TMIN measurements
daily_min_temp = daily_temp.filter(daily_temp["measurement_type"] == "TMIN")
# the 'measurement_type' column is no redundant. Drop it.
daily_min_temp = daily_min_temp.drop("measurement_type")
# the 'measurement_value' is the daily t_min
daily_min_temp = daily_min_temp.withColumnRenamed("measurement_value", "t_min")

daily_min_temp.show(5)

+-----------+----------+-----+
| station_id|      date|t_min|
+-----------+----------+-----+
|ITE00100554|1800-01-01| -148|
|EZE00100082|1800-01-01| -135|
|ITE00100554|1800-01-02| -125|
|EZE00100082|1800-01-02| -130|
|ITE00100554|1800-01-03|  -46|
+-----------+----------+-----+
only showing top 5 rows



In [74]:
# get historical min temp per station with a simple groupby
historical_min_temp = daily_min_temp.groupBy("station_id").agg(
    F.min("t_min").alias("t_min")
)
historical_min_temp.show()

+-----------+-----+
| station_id|t_min|
+-----------+-----+
|ITE00100554| -148|
|EZE00100082| -135|
+-----------+-----+



In [75]:
# We want to know which date corresponds to the historical t_min.
# To do this, perform an inner join with 'station_id' and 't_min' as keys
historical_min_temp = historical_min_temp.join(
    daily_min_temp, ["station_id", "t_min"], "inner"
)

historical_min_temp.show()

+-----------+-----+----------+
| station_id|t_min|      date|
+-----------+-----+----------+
|ITE00100554| -148|1800-01-01|
|EZE00100082| -135|1800-01-01|
+-----------+-----+----------+



In [76]:
spark.stop()